In [1]:
# %%
import pandas as pd
#import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
#import plotly.graph_objects as go
# 출력 제한 해제
# pd.set_option('display.max_columns',None)
# pd.set_option('display.max_rows',None)
# pd.set_option('display.max_seq_items', None)
# pd.options.display.max_colwidth = 100

import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
# %%
train = pd.read_csv('./data/train.csv')
test =  pd.read_csv('./data/test.csv')
trade = pd.read_csv('./data/international_trade.csv')

train.head()


,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0


In [3]:
item_dict = {
    '감귤' : 'TG',
    '꽃양배추와 브로콜리(broccoli)' : 'BC',
    '순무' : 'RD',
    '당근' : 'CR',
    '양배추' : 'CB'
}

item_list = ['감귤', '꽃양배추와 브로콜리(broccoli)', '순무', '당근', '양배추']
trade['item'] = [item_dict[e] if e in item_list else 'None' for e in trade['품목명']]

In [4]:
# %%
time = pd.to_datetime(train['timestamp'].copy())

train['Date'] = pd.to_datetime(train['timestamp'])
train['week'] = train['Date'].apply(lambda x: x.isocalendar()[1]) # 일요일 제거를 위함
train['day_name'] = train['Date'].dt.day_name()

train['year'] = train['timestamp'].apply(lambda x : int(x[0:4]))
train['month'] = train['timestamp'].apply(lambda x : int(x[5:7]))
train['day'] = train['timestamp'].apply(lambda x : int(x[8:10]))

trade['year'] = trade['기간'].apply(lambda x : int(x[0:4]))
trade['month'] = trade['기간'].apply(lambda x : int(x[5:7]))

In [5]:
result = train.groupby(['year', 'month', 'item'])['supply(kg)'].sum().reset_index()

trade = trade.merge(result, on=['year', 'month','item'], how='outer')
trade['전체 거래량'] = trade['수출 중량'] + trade['수입 중량']

In [6]:
train.shape

(59397, 13)

In [7]:
m_train = train.merge(trade, on=['year', 'month', 'item'], how='outer')
m_train = m_train.dropna(subset=['ID'])
m_train = m_train.fillna(0)

In [8]:
# %%
import datetime
import holidays

# %%
#train = train[(train['supply(kg)'] != 0) & (train['supply(kg)'] != 0)] # 이거차이도 큼!!! 거의 rmse 1200에서 680으로 줄어듦. 근데 이건 validation에만이고 test는 이거하면 정반대로됨 ㅠ...

# %%
#시계열 특성을 학습에 반영하기 위해 timestamp를 월, 일, 시간으로 나눕니다
test_re = test.copy()

test_re['year'] = test_re['timestamp'].apply(lambda x : int(x[0:4]))
test_re['month'] = test_re['timestamp'].apply(lambda x : int(x[5:7]))
test_re['day'] = test_re['timestamp'].apply(lambda x : int(x[8:10]))
test_re['Date'] = pd.to_datetime(test_re['timestamp'])
test_re['week'] = test_re['Date'].apply(lambda x: x.isocalendar()[1]) # 일요일 제거를 위함
test_re['day_name'] = test_re['Date'].dt.day_name()


# %%
test_re.head()

,ID,timestamp,item,corporation,location,year,month,day,Date,week,day_name
0,TG_A_J_20230304,2023-03-04,TG,A,J,2023,3,4,2023-03-04,9,Saturday
1,TG_A_J_20230305,2023-03-05,TG,A,J,2023,3,5,2023-03-05,9,Sunday
2,TG_A_J_20230306,2023-03-06,TG,A,J,2023,3,6,2023-03-06,10,Monday
3,TG_A_J_20230307,2023-03-07,TG,A,J,2023,3,7,2023-03-07,10,Tuesday
4,TG_A_J_20230308,2023-03-08,TG,A,J,2023,3,8,2023-03-08,10,Wednesday


In [9]:
m_test = test_re.merge(trade, on=['year', 'month', 'item'], how='outer')
m_test = m_test.dropna(subset=['ID'])
m_test = m_test.fillna(0)

In [10]:
m_train

,ID,timestamp,item,corporation,location,supply(kg)_x,price(원/kg),Date,week,day_name,...,day,기간,품목명,수출 중량,수출 금액,수입 중량,수입 금액,무역수지,supply(kg)_y,전체 거래량
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019-01-01,1.0,Tuesday,...,1.0,2019-01,감귤,58368.0,172.0,0.0,0.0,172.0,14704526.7,58368.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019-01-02,1.0,Wednesday,...,2.0,2019-01,감귤,58368.0,172.0,0.0,0.0,172.0,14704526.7,58368.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019-01-03,1.0,Thursday,...,3.0,2019-01,감귤,58368.0,172.0,0.0,0.0,172.0,14704526.7,58368.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019-01-04,1.0,Friday,...,4.0,2019-01,감귤,58368.0,172.0,0.0,0.0,172.0,14704526.7,58368.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019-01-05,1.0,Saturday,...,5.0,2019-01,감귤,58368.0,172.0,0.0,0.0,172.0,14704526.7,58368.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,BC_E_J_20230302,2023-03-02,BC,E,J,2984.0,3144.0,2023-03-02,9.0,Thursday,...,2.0,0,0,0.0,0.0,0.0,0.0,0.0,61472.0,0.0
59393,BC_E_J_20230303,2023-03-03,BC,E,J,4920.0,3045.0,2023-03-03,9.0,Friday,...,3.0,0,0,0.0,0.0,0.0,0.0,0.0,61472.0,0.0
59394,BC_E_S_20230301,2023-03-01,BC,E,S,2160.0,3816.0,2023-03-01,9.0,Wednesday,...,1.0,0,0,0.0,0.0,0.0,0.0,0.0,61472.0,0.0
59395,BC_E_S_20230302,2023-03-02,BC,E,S,1152.0,3321.0,2023-03-02,9.0,Thursday,...,2.0,0,0,0.0,0.0,0.0,0.0,0.0,61472.0,0.0


In [11]:
# %%
from sklearn.preprocessing import LabelEncoder
train_x = m_train.drop(columns=['ID', 'timestamp', 'supply(kg)_x', 'price(원/kg)','Date'])
train_y = m_train['price(원/kg)']
test_x  = m_test.drop(columns=['ID', 'timestamp','Date'])

In [12]:
qual_col = ['item', 'corporation', 'location','day_name']

train_x = pd.get_dummies(train_x, columns=qual_col, prefix=qual_col)
test_x  = pd.get_dummies(test_x, columns=qual_col, prefix=qual_col)

# for i in qual_col:
#     le = LabelEncoder()
#     train_x[i]=le.fit_transform(train_x[i])
#     test_x[i]=le.transform(test_x[i]) #test 데이터에 대해서 fit하는 것은 data leakage에 해당합니다

In [13]:
train_x = train_x.drop(columns=['기간','품목명'])
test_x  = test_x.drop(columns=['기간','품목명'])

test_x.columns = [e if e != 'supply(kg)' else 'supply(kg)_y' for e in test_x.columns.to_list()]

In [14]:
train_x = pd.concat([train_x, train_y], axis=1)

In [15]:
train_x.head()

,week,year,month,day,수출 중량,수출 금액,수입 중량,수입 금액,무역수지,supply(kg)_y,...,location_J,location_S,day_name_Friday,day_name_Monday,day_name_Saturday,day_name_Sunday,day_name_Thursday,day_name_Tuesday,day_name_Wednesday,price(원/kg)
0,1.0,2019,1,1.0,58368.0,172.0,0.0,0.0,172.0,14704526.7,...,1,0,0,0,0,0,0,1,0,0.0
1,1.0,2019,1,2.0,58368.0,172.0,0.0,0.0,172.0,14704526.7,...,1,0,0,0,0,0,0,0,1,0.0
2,1.0,2019,1,3.0,58368.0,172.0,0.0,0.0,172.0,14704526.7,...,1,0,0,0,0,0,1,0,0,1728.0
3,1.0,2019,1,4.0,58368.0,172.0,0.0,0.0,172.0,14704526.7,...,1,0,1,0,0,0,0,0,0,1408.0
4,1.0,2019,1,5.0,58368.0,172.0,0.0,0.0,172.0,14704526.7,...,1,0,0,0,1,0,0,0,0,1250.0


In [16]:
train_x.corr()['price(원/kg)'].sort_values()

전체 거래량               -0.236542
item_RD              -0.236393
수입 금액                -0.227445
day_name_Sunday      -0.226477
수입 중량                -0.223645
item_CR              -0.197818
item_CB              -0.185041
corporation_F        -0.099938
week                 -0.080900
supply(kg)_y         -0.079610
month                -0.076673
item_BC              -0.059762
수출 중량                -0.057893
corporation_E        -0.047335
location_J           -0.040949
corporation_D        -0.026650
수출 금액                -0.013909
corporation_A         0.002705
day                   0.011755
day_name_Saturday     0.012714
year                  0.029442
location_S            0.040949
day_name_Wednesday    0.041550
day_name_Thursday     0.041971
day_name_Friday       0.042265
day_name_Monday       0.043725
day_name_Tuesday      0.043927
corporation_C         0.070734
corporation_B         0.078032
무역수지                  0.199215
item_TG               0.591802
price(원/kg)           1.000000
Name: pr

In [17]:
low_corr = ['week', 'month', 'day', 'year']

In [23]:
# train_x = train_x.drop(columns=low_corr)
# test_x = test_x.drop(columns=low_corr)

In [18]:
# import pycaret regression and init setup
from pycaret.regression import *

s = setup(train_x, target = 'price(원/kg)', 
        session_id = 1004, 
        fold = 10, 
        normalize = True, 
        normalize_method = 'zscore', 
        train_size=0.8) ##nomalize에 robust, zscore 가능
# normalise를 robust, zscore로 하면 rmse 너무 높아짐 (1200 수준). minmax 가 680 수준
#fole 수나 train_size를 바꿔보자

# %%
best_model = compare_models()


,Description,Value
0,Session id,1004
1,Target,price(원/kg)
2,Target type,Regression
3,Original data shape,"(59397, 32)"
4,Transformed data shape,"(59397, 32)"
5,Transformed train set shape,"(47517, 32)"
6,Transformed test set shape,"(11880, 32)"
7,Numeric features,31
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,303.0325,741460.9416,860.8195,0.8197,2.1444,0.1844,1.9420
catboost,CatBoost Regressor,399.6061,756054.2663,869.3793,0.8162,3.6733,0.2294,1.3640
xgboost,Extreme Gradient Boosting,388.8510,757522.7750,870.1182,0.8158,3.5877,0.2239,0.1540
lightgbm,Light Gradient Boosting Machine,386.7708,761089.5907,872.3074,0.8150,3.3992,0.2380,0.3390
et,Extra Trees Regressor,314.0310,862981.1797,928.3658,0.7903,2.0176,0.1989,1.7060
gbr,Gradient Boosting Regressor,552.9511,1101827.5847,1049.4200,0.7323,4.1228,0.2853,1.1420
knn,K Neighbors Regressor,483.8680,1142720.9750,1068.6636,0.7224,2.5591,0.3358,0.2120
dt,Decision Tree Regressor,366.1037,1331409.2430,1153.2220,0.6765,2.0566,0.2514,0.0730
ridge,Ridge Regression,1026.2294,2098835.3991,1448.6303,0.4901,5.0063,0.5643,0.0370
br,Bayesian Ridge,1026.2297,2098838.2078,1448.6313,0.4901,5.0063,0.5643,0.0510


In [19]:
reg  = create_model('xgboost')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,306.5889,724936.4634,851.4320,0.8178,2.1340,0.1921
1,296.4587,723734.7862,850.7260,0.8194,2.2044,0.1746
2,312.5333,743954.0749,862.5277,0.8186,2.1940,0.1935
3,281.3229,652879.7897,808.0098,0.8498,2.0793,0.1862
4,306.8911,787602.8795,887.4699,0.8096,2.1537,0.1779
5,310.4114,780796.5083,883.6269,0.8110,2.1556,0.1821
6,286.5255,733947.5583,856.7074,0.8115,2.0400,0.1741
7,305.9471,747836.1053,864.7752,0.8175,2.1760,0.1862
8,307.8715,752141.7538,867.2611,0.8165,2.1501,0.1896


In [20]:
tuned_clf = tune_model(reg, optimize = 'R2')
predictions = predict_model(tuned_clf, data = test_x)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,371.8575,817845.6805,904.3482,0.7986,2.5642,0.2359
1,405.9128,952131.1378,975.7721,0.7807,2.6478,0.2394
2,382.1366,852691.2792,923.4128,0.7704,2.6533,0.2481
3,389.1678,853284.8515,923.7342,0.7821,2.6919,0.2429
4,371.5445,797376.4065,892.9594,0.8150,2.5812,0.2329
5,406.9775,912719.3246,955.3635,0.7824,2.6403,0.2365
6,387.7355,884768.8901,940.6215,0.7824,2.6476,0.2360
7,383.5733,877848.1871,936.9355,0.7934,2.6351,0.2328
8,385.5864,858184.6686,926.3826,0.7939,2.5978,0.2471


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [38]:

result = pd.concat([test_re, predictions['prediction_label']], axis = 1)

result['answer'] = result['prediction_label'].copy()

del result['timestamp']
del result['item']
del result['corporation']
del result['location']
del result['year']
del result['month']
del result['day']
del result['Date']
del result['week']
del result['day_name']
del result['prediction_label']

In [40]:
for i in range(len(result)):
    if (i-1) % 7 == 0: # 1, 8, 15, 22
        result.iloc[i,-1] = 0

result.head(20)

,ID,answer
0,TG_A_J_20230304,3922.94
1,TG_A_J_20230305,0.00
2,TG_A_J_20230306,3751.01
3,TG_A_J_20230307,3765.07
4,TG_A_J_20230308,3818.61
5,TG_A_J_20230309,3700.83
6,TG_A_J_20230310,3667.19
7,TG_A_J_20230311,3765.61
8,TG_A_J_20230312,0.00
9,TG_A_J_20230313,3705.77


In [42]:
result.to_csv('result4_tuned.csv', index = False)